In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [12]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [13]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
#username = 'i9o5yf5bh59gq1nttz2iic006'
scope = spotify_creds['top_artist_scope']
redirect_uri = spotify_creds['top_artist_redirect_url']

In [ ]:
spotify_creds['saved_library_token']

In [16]:
which_token = 'saved_library'
str1 = "%s_token" % (which_token)

In [18]:
spotify_creds[str1]

'BQDs3l4g6Gm5JFXDE4C7ngKQ9Z_7fMFlkN0oL5WwlTT7L63HjMppDNLd_Ml7ZmrwoS6zjBrsEg0-2Wr0qGHB0ACrsd5P2fMvcmayL1vhiu3wXRVXgjR2IkMgB-60EPKOoyXkZCssUb8CwJlC2CYiYPffftUK_-yRtzAxTbZwKauGf1yJ1Kk'

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [9]:
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [10]:
token

'BQDWJYYXNHAECjcq9XQPWxBjZ7BVOBymuoYcCl2bI-v4lOrgv8FaklI5j7x9tSA0FiBdMz6lN1T754BKkdYq71IqnMb5sNTdqRH6Gv8n-GMsD3VkKFs2yLPR2veNTMrO9fMspTC2rDOvOIFVMoW7O9QZVJWk8ynhH5TbZxWGwSiBdQ'

In [6]:
token = 'BQDs3l4g6Gm5JFXDE4C7ngKQ9Z_7fMFlkN0oL5WwlTT7L63HjMppDNLd_Ml7ZmrwoS6zjBrsEg0-2Wr0qGHB0ACrsd5P2fMvcmayL1vhiu3wXRVXgjR2IkMgB-60EPKOoyXkZCssUb8CwJlC2CYiYPffftUK_-yRtzAxTbZwKauGf1yJ1Kk'

In [7]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [15]:
sp.current_user()

{'display_name': 'Nihw',
 'external_urls': {'spotify': 'https://open.spotify.com/user/6oup6y9z3si0zmr49hq1kze94'},
 'followers': {'href': None, 'total': 1},
 'href': 'https://api.spotify.com/v1/users/6oup6y9z3si0zmr49hq1kze94',
 'id': '6oup6y9z3si0zmr49hq1kze94',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85f9b35ae5a6c28f047e6cc6c0',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:6oup6y9z3si0zmr49hq1kze94'}

In [13]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [7]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [8]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [9]:
df = get_saved_library()

In [20]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri, which_token):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    #token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    str1 = "%s_token" % (which_token)
    token = spotify_creds[str1]

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri, 'saved_library')